In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
import spacy;
from spacy.tokens import Span, Doc, Token
from spacy.matcher import PhraseMatcher
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA

import  matplotlib.pyplot  as plt
#import gensim
import numpy as np
from rcare.readutils import DataFrameUtils as Utils
from collections import namedtuple, defaultdict

In [3]:
nlp = spacy.load('en_core_web_sm')


In [4]:
DATA_PATHS = []
#DATA_PATH = 'data/lexpredict-contraxsuite-samples/cca_2011_Q3/text/'
#DATA_PATHS.append(DATA_PATH)
#df_2011_q3 = Utils.read_data(path = DATA_PATH)

DATA_PATH = 'data/lexpredict-contraxsuite-samples/cca_2011_Q3/text/'
#df_2011_q4 = Utils.read_data(path = DATA_PATH)
DATA_PATHS.append(DATA_PATH)

#DATA_PATH = 'data/lexpredict-contraxsuite-samples/cca_2012_Q4/text/'
#DATA_PATHSdf_2012_q4 = Utils.read_data(path = DATA_PATH)
#DATA_PATHS.append(DATA_PATH)

DATA_PATHS

SectionMap = namedtuple('SectionMap', ['name', 'keywords']);
SpacyDoc = namedtuple('SpacyDocs', ['name', 'doc']);
SpacySectionDoc = namedtuple('SpacySectionDoc', ['match_id', 'title', 'section', 'formatted_title']);

In [5]:
'''
matcher = PhraseMatcher(nlp.vocab)
sectionHeadings = ['CARD AGREEMENT', 'Definitions', 'Your Account', 'Authorized Users', 'Joint Accounts'
                   ,'Credit Limit','Billing Statement.'
                    ,'APRs\n', 'Promotions', 'Interest Charges Based on APRs',
                   'Fees','Information on Foreign Currency Conversion','Payments',
                   'Credit Reporting','Information Sharing','Changes to this Agreement','Default',
                   #'Refusal of the Card, Closed Accounts,\nand Related Provisions',
                  'Refusal of the Card,', #'Lost or Stolen Cards or Account Numbers.', 'Closing Your Account.'
                   'ARBITRATION',
                   'Claims Covered', 'How Arbitration Works', 'Survival and Severability of Terms', 
                   'Governing Law and Enforcing our Rights', 'Your Billing Rights']
patterns = [nlp(text) for text in sectionHeadings]

matcher.add('sectionHeadings', None, *patterns)
'''

docs = []
for path in DATA_PATHS:
    fileData_g = Utils.read_data_gen(path = path, numfiles = 2)    
    for idx, fileData in enumerate(fileData_g):
        text = fileData.text
        name = fileData.name
        docs.append(SpacyDoc(name, nlp(text)))
    


2018-08-06 16:29:20,287 : INFO : Number of files 720 in data/lexpredict-contraxsuite-samples/cca_2011_Q3/text/
2018-08-06 16:29:20,294 : INFO : Reading file creditcardagreement_1000.TXT
2018-08-06 16:29:21,060 : INFO : Reading file creditcardagreement_1043.TXT
2018-08-06 16:29:21,536 : INFO : Reading file data done..


In [ ]:
doc = docs[1].doc
matches = matcher(doc)
prev = (-1, -1, nlp("-1"))
spans = []
for match_id, start, end in matches:
    #span = doc[prev[1]:start-1]
    if(prev[0] == -1 ):
        spans.append(SpacySectionDoc("first", "dummy"  ,get_lemma(doc[:start]), Utils.format_title("dummy"))) 
    else:
        spans.append(SpacySectionDoc(match_id, prev[2] , get_lemma(doc[prev[1]:start-1]), Utils.format_title(prev[2].text)))
        
    prev = (start, end, doc[start:end])
    
spans.append(SpacySectionDoc("last", prev[2] , get_lemma(doc[prev[1]:]), Utils.format_title(prev[2].text)))    

In [ ]:
def get_lemma (spacy_doc):
    tokens = []    
    for w in spacy_doc:
        if w.text.strip() not in ('\n', '$','\n\n' , '\n\n\n', '\n\n\n\n',''  ) \
        and not 'â' in w.text \
        and w.lemma_ != '-PRON-'\
        and not w.is_stop and not w.is_punct:
            # we add the lematized version of the word
            #logging.info("{} - oov {}".format(w.text, w.is_oov))
            #tokens.append(w.text.lower())    
            #tokens.append(w.text)  
            tokens.append(w.lemma_)  
    return " ".join(tokens)

In [ ]:
section_map = {
    'account': ['Using the Account', 'Your Account.'],
    'billing': ['Billing Statement.', 'Finance Charges'],
    'apr': ['Rate Changes', 'APRs'],
    'payments': ['Monthly Payment', 'Payments'],
    'default': ['Default.', 'Default'],
    'foreign_transactions' = ['Foreign Transactions', 'Information on Foreign Currency Conversion'],
    'credit_reporting' : [ 'Credit Reporting', 'Credit Information']
}



In [6]:
doc = docs[0].doc
sents = list(doc.sents)

In [11]:
list(enumerate(sents[:30]))

#sents[9].as_doc().to_array([LOWER, POS, ENT_TYPE, IS_ALPHA])
#sents[9].text

[(0, MASTERCARD AND VISA LINE OF CREDIT AGREEMENT
  ),
 (1, The purpose of the Agreement is to establish the terms and conditions of
  your credit card agreement with McCoy Federal Credit Union.),
 (2, In this
  agreement, the words "you" and "your" mean any person receiving and using
  the card and that by using the card for the first time),
 (3, "you" agree to all terms
  and conditions in this agreement. "),
 (4, Card" means a Mastercard or VISA credit
  card and any duplicates and renewals the Credit Union issues. "),
 (5, Account"
  means your Mastercard or VISA credit card line of credit account with the
  Credit Union.),
 (6, "Credit Union," "we," "us," and "our" mean McCoy Federal
  Credit Union.),
 (7, You must be a member of the Credit Union to apply for a credit
  card.),
 (8,
  Except to the extent that Federal Law is applicable, the validity, construction
  and enforcement of the Agreement and all matters arising out of the issuance
  and use of the Card shall be governed 

In [71]:
for token in doc[:50]:
    print(token.text, "==>",token.dep_, token.head.text, token.head.pos_,
          token.is_upper)
    

MASTERCARD ==> ROOT MASTERCARD PROPN True
AND ==> cc MASTERCARD PROPN True
VISA ==> compound LINE NOUN True
LINE ==> conj MASTERCARD PROPN True
OF ==> prep LINE NOUN True
CREDIT ==> compound AGREEMENT PROPN True
AGREEMENT ==> pobj OF ADP True


 ==>  AGREEMENT PROPN False
The ==> det purpose NOUN False
purpose ==> nsubj is VERB False
of ==> prep purpose NOUN False
the ==> det Agreement PROPN False
Agreement ==> pobj of ADP False
is ==> ROOT is VERB False
to ==> aux establish VERB False
establish ==> xcomp is VERB False
the ==> det terms NOUN False
terms ==> dobj establish VERB False
and ==> cc terms NOUN False
conditions ==> conj terms NOUN False
of ==> prep terms NOUN False

 ==>  of ADP False
your ==> poss agreement NOUN False
credit ==> compound card NOUN False
card ==> compound agreement NOUN False
agreement ==> pobj of ADP False
with ==> prep agreement NOUN False
McCoy ==> compound Union PROPN False
Federal ==> compound Union PROPN False
Credit ==> compound Union PROPN False
Union

In [66]:
from spacy.symbols import nsubj, VERB
verbs = set()
for possible_subject in sents[29]:
    print (possible_subject, possible_subject.head, list(possible_subject.ancestors), possible_subject.is_sent_start)
    if possible_subject.dep == nsubj and possible_subject.head.pos == VERB:
        verbs.add(possible_subject.head)
print(verbs)

The Balance [Balance, determined] True
Average Balance [Balance, determined] None
Daily Balance [Balance, determined] None
Balance determined [determined] None

 Balance [Balance, determined] None
is determined [determined] None
determined determined [] None
as follows [follows, determined] None
follows determined [determined] None
: determined [determined] None
set()


In [38]:
list(stop_words)

[('other', 'o'),
 ('two', 't'),
 ('she', 's'),
 ('alone', 'a'),
 ('hers', 'h'),
 ('enough', 'e'),
 ('becoming', 'b'),
 ('amount', 'a'),
 ('himself', 'h'),
 ('such', 's'),
 ('sometime', 's'),
 ('noone', 'n'),
 ('though', 't'),
 ('thereupon', 't'),
 ('wherever', 'w'),
 ('will', 'w'),
 ('now', 'n'),
 ('therefore', 't'),
 ('forty', 'f'),
 ('name', 'n'),
 ('whom', 'w'),
 ('often', 'o'),
 ('unless', 'u'),
 ('this', 't'),
 ('whether', 'w'),
 ('nothing', 'n'),
 ('well', 'w'),
 ('along', 'a'),
 ('from', 'f'),
 ('on', 'o'),
 ('should', 's'),
 ('hundred', 'h'),
 ('much', 'm'),
 ('seems', 's'),
 ('wherein', 'w'),
 ('beyond', 'b'),
 ('used', 'u'),
 ('eleven', 'e'),
 ('you', 'y'),
 ('except', 'e'),
 ('so', 's'),
 ('top', 't'),
 ('even', 'e'),
 ('without', 'w'),
 ('give', 'g'),
 ('and', 'a'),
 ('whoever', 'w'),
 ('about', 'a'),
 ('nor', 'n'),
 ('which', 'w'),
 ('together', 't'),
 ('an', 'a'),
 ('everyone', 'e'),
 ('below', 'b'),
 ('itself', 'i'),
 ('doing', 'd'),
 ('mostly', 'm'),
 ('many', 'm'),
 ('